In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Import from the CRUD file
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to database via CRUD Module 
db = AnimalShelter()

# Read all records from the database
df = pd.DataFrame.from_records(db.read({}))

# Drop MongoDB’s '_id' column to avoid ObjectID issues in DataTable
df.drop(columns=['_id'], inplace=True, errors='ignore')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare logo
image_filename = 'Logo.png'  # ensure this image exists in the project folder
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'80px'}),
        html.B(html.H1('Grazioso Salvare Dashboard - Tashyra Adams'))  # unique identifier
    ]),
    html.Hr(),

    # Interactive filtering options for Rescue Type
    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Wilderness Rescue', 'value': 'wilderness'},
                {'label': 'Disaster/Tracking', 'value': 'disaster'}
            ],
            value='all',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': '#f2f2f2', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),

    # Charts and Map side by side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter DataTable based on Rescue Type selection
@app.callback(
    Output('datatable-id','data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    if filter_type == 'all':
        return df.to_dict('records')
    filtered_df = df.copy()
    if filter_type == 'water':
        filtered_df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}
        }))
    elif filter_type == 'wilderness':
        filtered_df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}
        }))
    elif filter_type == 'disaster':
        filtered_df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog",
            "age_upon_outcome_in_weeks":{"$gte":26, "$lte":156}
        }))
    filtered_df.drop(columns=['_id'], inplace=True, errors='ignore')
    return filtered_df.to_dict('records')

# Pie chart for breed distribution of filtered data
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    if not viewData:
        return html.P("No data to display")
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Breed Distribution of Selected Dogs')
        )
    ]

# Highlight selected columns in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Geo-map for selected dog
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return html.P("Waiting for data...")
    dff = pd.DataFrame.from_dict(viewData)
    if not index or len(index) == 0:
        row = 0
    else:
        row = index[0]
    try:
        lat = float(dff.iloc[row].get('location_lat', 30.75))
        lon = float(dff.iloc[row].get('location_long', -97.48))
    except Exception:
        lat, lon = 30.75, -97.48
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row].get('name', 'Unknown')),
                dl.Popup([
                    html.H4(f"Name: {dff.iloc[row].get('name', 'Unknown')}"),
                    html.P(f"Breed: {dff.iloc[row].get('breed', 'N/A')}"),
                    html.P(f"Animal Type: {dff.iloc[row].get('animal_type', 'N/A')}")
                ])
            ])
        ])
    ]

# Run the app in JupyterLab mode (external browser)
app.run_server(mode='external', port=8050)


Dash app running on https://jasonbuffalo-togadoor-3000.codio.io/proxy/8050/
